In [ ]:
import subprocess
import json

def get_repo():
    """Auto-detect current GitHub repo (owner/repo format)"""
    result = subprocess.run(
        ["gh", "repo", "view", "--json", "nameWithOwner", "--jq", ".nameWithOwner"],
        capture_output=True, text=True, check=True
    )
    return result.stdout.strip()

def get_open_issue_numbers(repo):
    """Get all open issue numbers, skipping pull requests"""
    try:
        result = subprocess.run(
            [
                "gh", "issue", "list",
                "--repo", repo,
                "--state", "open",  # Only fetch open issues
                "--limit", "1000",
                "--json", "number,title,labels,assignee,state",
                "--jq", '.[] | select(.state == "open") | .number'  # Only select open issues
            ],
            capture_output=True, text=True, check=True
        )
        
        # Split the result by lines and filter out empty lines (if any)
        issue_numbers = result.stdout.strip().splitlines()
        return [issue_number.strip() for issue_number in issue_numbers if issue_number.strip()]
    except subprocess.CalledProcessError as e:
        print(f"Error: Failed to fetch open issues for repo '{repo}'.\n{e.stderr}")
        return []

def get_issue_details(repo, issue_number):
    """Fetch detailed information for a single issue"""
    try:
        result = subprocess.run(
            [
                "gh", "issue", "view", str(issue_number),
                "--repo", repo,
                "--json", "number,title,labels,comments,linkedPullRequests"
            ],
            capture_output=True, text=True, check=True
        )
        return json.loads(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Warning: Failed to fetch Issue #{issue_number}. Skipping.\n{e.stderr}")
        return None


SyntaxError: expected 'except' or 'finally' block (4285438454.py, line 32)

In [11]:
repo = get_repo()
issue_numbers = get_open_issue_numbers(repo)
all_issues = []

for issue in issue_numbers:
    print(f"Fetching details for Issue #{issue}")
    details = get_issue_details(repo, issue)

    if details is None:
        continue  # Skip issues that couldn't be fetched

    # Extract labels (names only)
    labels = [label['name'] for label in details.get('labels', [])]

    # Extract linked PR URLs
    # pull_requests = [pr['url'] for pr in details.get('linkedPullRequests', [])]

    # Extract comments
    comments = []
    for comment in details.get('comments', []):
        comments.append({
            "author": comment.get("author", {}).get("login"),
            "body": comment.get("body"),
            "createdAt": comment.get("createdAt")
        })

    # Assemble final issue record
    all_issues.append({
        "issue": details.get("number"),
        "title": details.get("title"),
        "labels": labels,
        "pull_requests": pull_requests,
        "comments": comments
    })

# Save all issues to a JSON file
with open("issues_with_comments.json", "w") as f:
    json.dump(all_issues, f, indent=2)

print(f"\n✅ Saved {len(all_issues)} issues to issues_with_comments.json")


Error fetching issue list: Unknown JSON field: "isPullRequest"
Available fields:
  assignees
  author
  body
  closed
  closedAt
  comments
  createdAt
  id
  labels
  milestone
  number
  projectCards
  projectItems
  reactionGroups
  state
  title
  updatedAt
  url



TypeError: 'NoneType' object is not iterable